In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.python.keras.callbacks import TensorBoard
#import tensorflow_addons as tfa
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
from time import time
import os
from sklearn.utils.class_weight import compute_class_weight
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

In [2]:
#read train dataset
tf.random.set_seed(5)
print("open dataset")
train = pd.read_csv("../dataset/goodreads_train.csv")

open dataset


In [3]:
# Load a np_archive of review_text col of train dataset preprocess in main.py
train_prepro = pd.DataFrame(data=np.load(file="../vocabulaires/prepro_train_archive_NEG_lem.csv.npy", allow_pickle=True), columns=['review_text'])['review_text']


In [4]:
# add review text col to train dataset
train['review_text'] = train_prepro

In [5]:
l2_rate = 0.00005
dropout_rate = 0.05
rating = keras.utils.to_categorical(train['rating'], num_classes=6)
#create model layers
inputs = keras.Input(shape=(1,), dtype=tf.string) # text
inputs2 = keras.Input(shape=1, dtype=tf.float32) # n_comment
inputs3 = keras.Input(shape=1, dtype=tf.float32) # n_votes
#create vectorize layer, to transform words in integer
vectorize_layer = keras.layers.TextVectorization(
    standardize='lower_and_strip_punctuation',
    split='whitespace',
    output_mode='int',
    output_sequence_length=1400,
    vocabulary=np.load('../vocabulaires/voc_lemm_without_NP_NEG.npy')
)(inputs)

embedding = keras.layers.Embedding(453118, 600, input_length=1400, batch_size=100,embeddings_regularizer=tf.keras.regularizers.l2(l2_rate))(vectorize_layer)
print(embedding)
flatten = keras.layers.Flatten()(embedding)

conc = keras.layers.concatenate([flatten, inputs2,inputs3])
layer2 = keras.layers.Dense(700, activation=tf.keras.activations.relu,kernel_regularizer=tf.keras.regularizers.l2(l2_rate),bias_regularizer=tf.keras.regularizers.l2(l2_rate))(conc)
drop1 = keras.layers.Dropout(dropout_rate)(layer2)
layer3 = keras.layers.Dense(300, activation=tf.keras.activations.tanh,kernel_regularizer=tf.keras.regularizers.l2(l2_rate),bias_regularizer=tf.keras.regularizers.l2(l2_rate))(drop1)
drop2 = keras.layers.Dropout(dropout_rate)(layer3)
outputs = keras.layers.Dense(6, activation=tf.keras.activations.sigmoid)(drop2)

KerasTensor(type_spec=TensorSpec(shape=(None, 1400, 600), dtype=tf.float32, name=None), name='embedding/embedding_lookup/Identity_1:0', description="created by layer 'embedding'")


In [6]:
model = keras.Model(inputs=[inputs, inputs2, inputs3], outputs=outputs, name="mnist_model")
tensorboard = TensorBoard(log_dir="logs".format(time()))

In [7]:
#for learning_rate in learning_rates:
model.compile(optimizer=tf.keras.optimizers.Adamax(),
              loss=tf.keras.losses.binary_crossentropy,
              metrics=[
                  tf.keras.metrics.categorical_accuracy
                       ], run_eagerly=False
              )

In [8]:
class_weights = compute_class_weight(class_weight='balanced',classes= np.unique(train['rating']), y = train['rating'])
di = {}
for i in range(len(class_weights)):
    di[i] = class_weights[i]
#model.save("../models_trained/pmc8_3")

In [9]:
model.fit([train['review_text'], train['n_comments'], train['n_votes']], rating, epochs=5,
                  callbacks=[
                      tf.keras.callbacks.TensorBoard(log_dir="logs"),
                  ],
                  batch_size=800, class_weight=di, shuffle=True, validation_split=0.1
                  )

Epoch 1/5


C:\Users\Carmo\miniconda3\envs\goodBook\lib\site-packages\tensorflow\python\framework\indexed_slices.py:436: UserWarning: Converting sparse IndexedSlices to a dense Tensor with 271870800 elements. This may consume a large amount of memory.
  warnings.warn(


 245/1013 [======>.......................] - ETA: 2:00:05 - loss: 1.4012 - categorical_accuracy: 0.1687

InvalidArgumentError: Graph execution error:

Detected at node 'mnist_model/embedding/embedding_lookup' defined at (most recent call last):
    File "C:\Users\Carmo\miniconda3\envs\goodBook\lib\runpy.py", line 196, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "C:\Users\Carmo\miniconda3\envs\goodBook\lib\runpy.py", line 86, in _run_code
      exec(code, run_globals)
    File "C:\Users\Carmo\miniconda3\envs\goodBook\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "C:\Users\Carmo\miniconda3\envs\goodBook\lib\site-packages\traitlets\config\application.py", line 846, in launch_instance
      app.start()
    File "C:\Users\Carmo\miniconda3\envs\goodBook\lib\site-packages\ipykernel\kernelapp.py", line 712, in start
      self.io_loop.start()
    File "C:\Users\Carmo\miniconda3\envs\goodBook\lib\site-packages\tornado\platform\asyncio.py", line 215, in start
      self.asyncio_loop.run_forever()
    File

In [ ]:
#model.save("../models_trained/pmc8_3_train")

In [ ]:
#read test dataset
test = pd.read_csv("../dataset/goodreads_test.csv")

In [11]:
# Load a np_archive of review_text col of train dataset preprocess in main.py
test_prepro = pd.DataFrame(data=np.load(file="../vocabulaires/prepro_test_archive_NEG.csv.npy", allow_pickle=True), columns=['review_text'])['review_text']
test['review_text'] = test_prepro

In [ ]:
# test the model with test dataset

In [12]:
restest = model.predict([test['review_text'], test['n_comments'], test['n_votes']])

14939/14939 [==============================] - 92s 6ms/step


In [13]:
# reverse keras.utils.to_categorical for kaggle submission
ff = []
for line in tqdm(restest):
    tmp = -2
    category = None
    for i in (range(6)):
        if line[i] > tmp:
            category = i
            tmp = line[i]
    ff.append(category)
data = np.array(ff)

  0%|          | 0/478033 [00:00<?, ?it/s]

In [14]:
test['rating'] = data

In [15]:
data

array([4, 2, 5, ..., 4, 5, 5])

In [16]:
# create a dataframe for kaggle
id = test['review_id'].to_numpy()
rating = test['rating'].to_numpy()
df = pd.DataFrame( columns=['review_id', 'rating'])

In [17]:
df['review_id'] = id
df['rating'] = rating

In [18]:
# create a csv for submission
df.to_csv('pmc8_3_model.csv',index=False)